### Libraries

In [1]:
import sys
sys.path.insert(0,'/Users/jarad/Fake Folder/Python Libraries')

from jb_libraries import *
%matplotlib inline
from part_categories_data import *
from part_cost_data import *


== part_categories start ==


your df is called "all_cats_df"


== part_categories end ==


== part_cost_data start ==

0 nulls
the mean_stripe_margin is 77.44%

the part types with no MSRP are/is: ['sku']

avg gross profit for:
parts, combos, and stripes
which have been purchased within the last year
whose sku_status equals "working"
whose msrp is greater than zero
which were bought by non resellers
is 56.01%

the parts with negative gross profit are: [2885, 3400]

your dfs are: all_cost (cost on the sku level) and all_cost_by_part (cost on the part level)

== part_cost_data end ==



### Script settings

In [2]:
close_workbook = 'yes'

now = str((dt.datetime.now()).date())
print('as of %s' % now)

csv_path = '/Users/jarad/Fake Folder/Purchasing/Recurring/More Than 90 Days of Stock Report/CSVs/'

as of 2019-02-01


### Get data

In [3]:
skus_main = pd.read_sql(
'''
SELECT
timestamp,
k.sku,
s.sku_id,
k.part_id,
k.sku_name AS name,
actual_stock,
s.combined_sales_90 AS daily_sales,
s.sku_status AS status
FROM sku_stock_snapshots s
LEFT JOIN skus k ON s.sku_id = k.sku_id
WHERE DATE(timestamp) = ' '''+ now +''' '
''', db)

col_fix(skus_main)

In [4]:
parts_main = pd.read_sql(
'''
SELECT
pss.timestamp,
0 AS sku,
0 AS sku_id,
pss.part_id,
pd.products_name AS name,
pss.actual_stock,
pss.combined_sales_90 AS daily_sales,
LOWER(pss.discontinue_status) AS status
FROM parts_stock_snapshots pss
JOIN products_description pd ON pss.part_id = pd.part_id
WHERE DATE(pss.timestamp) = ' '''+ now +''' '
AND pss.part_id NOT IN '''+ str(tuple(skus_main['part id'])) +'''
''', db)

col_fix(parts_main)

In [5]:
combos_main = pd.read_sql(
'''
SELECT
part_id
FROM products_to_stuff
WHERE part_id > 0
''', db)

col_fix(combos_main)

In [6]:
meta_types = pd.read_sql(
'''
SELECT
part_id,
LOWER(products_meta_type) AS products_meta_type
FROM parts
''', db)

col_fix(meta_types)

In [7]:
super_main = pd.concat([skus_main, parts_main], sort = False)

# keep only the most recent entries
super_main.sort_values(['sku id','part id','timestamp'], inplace = True)
super_main.drop_duplicates(['sku id','part id'], keep = 'last', inplace = True)

# if negative make zero
for col in ['actual stock','daily sales']:
    super_main[col] = np.where(super_main[col] < 0, 0, super_main[col])
    
# calculate these
super_main['quarter sales'] = super_main['daily sales'] * 90    

super_main['days of stock'] = np.round(super_main['actual stock']/np.round(super_main['daily sales'], 2))
super_main['days of stock'].fillna(0, inplace = True)
super_main.drop('daily sales',1,inplace = True)
super_main.replace(np.inf, 0, inplace = True)    

new_cols = ['timestamp',
             'sku',
             'sku id',
             'part id',
             'name',
             'actual stock',
             'quarter sales',
             'status',
             'days of stock']

old_cols = super_main.columns

s1 = set(new_cols)
s2 = set(old_cols)

if len(s1.symmetric_difference(s2)) == 0:
    super_main = super_main[new_cols]
else:
    print('mismatch')
    print(s1.symmetric_difference(s2))
    raise 0
    
# part categories
super_main['category'] = super_main['part id'].map(dict(zip(all_cats_df['part id'], all_cats_df['cat name'])))

def cat_fix(x):
    if pd.isnull(x['category']):
        if x['status'] == 'obselete':
            return 'obselete'
        elif 'Strip' in x['name']:
            return 'strip'
        else:
            return 'no category'
    else:
        return x['category']

super_main['category'] = super_main.apply(cat_fix, axis = 1)
super_main['category'] = [x.lower() for x in super_main['category']]

# flag combos
super_main['combo'] = np.where(super_main['part id'].isin(combos_main['part id']), 'yes','no')

# flag meta types
super_main['meta type'] = super_main['part id'].map(dict(zip(meta_types['part id'], meta_types['products meta type'])))
# if it's a sku, give it meta type = sku
super_main['meta type'] = np.where((super_main['meta type'].isnull()) & (super_main['part id'] == 0),
                                  'sku', super_main['meta type'])
# lowercase
super_main['name'] = [x.lower() for x in super_main['name']]

### Get latest cost

In [8]:
super_main['tuple'] = list(zip(super_main['sku id'], super_main['part id']))

# all_cost comes from part_cost.py
all_cost['tuple'] = list(zip(all_cost['sku id'], all_cost['part id']))

super_main = pd.merge(super_main, all_cost[['tuple','latest cost']], on = 'tuple')
super_main.drop('tuple', 1, inplace = True)

super_main.rename(columns = {'latest cost':'latest cost per unit'}, inplace = True)
super_main['cost of inventory'] = super_main['latest cost per unit'] * super_main['actual stock']

### Restrict by stock

In [9]:
main = super_main[super_main['actual stock'] > super_main['quarter sales']].copy()

In [10]:
for col in main.columns:
    if main[col].dtype in ['float64','int64']:
        if np.sum(main[col] < 0) > 0:
            raise ValueError('{} has a negative value'.format(col))
        else:
            pass
            
print('\n')            

for col in main.columns:
    if main[col].dtype in ['float64','int64']:
        if np.sum(np.isinf(main[col])) > 0:
            raise ValueError('{} has an infinity'.format(col))
        else:
            pass

### Divide data

In [11]:
partnerships = pd.read_sql(
'''
SELECT
part_id
FROM products_description
WHERE products_name LIKE '%partner%'
''', db)

col_fix(partnerships)

partnerships_ls = partnerships.values.flatten().tolist()

In [12]:
ls1 = ['badge','patch','gift certificate','plushie','shirt']
ls2 = '|'.join(ls1)

def label(x):
    if x['part id'] > 0:
        if x['combo'] == 'no':
            if ((x['status'] == 'pending' 
                or 'badge' in x['name'] 
                or 'patch' in x['name'] 
                or 'gift certificate' in x['name']
                or 'plushie' in x['name'] 
                or 'shirt' in x['name']) 
            or ('sticker' in x['name'] 
                and ('accessories' in x['category'] 
                or '3d printing' in x['category']))
            or x['part id'] in partnerships_ls):
                label = 'not real'
            else:
                label = 'part'
        else:
            label = 'combo'
    else:
        label = 'sku'
        
    return label
        
main['label'] = main.apply(label, axis = 1)

# results
print('function results')
print(main['label'].value_counts())

# store dfs
dict_ = {}
for l in list(set(main['label'])):
    df = main[main['label'] == l].copy()
    df.sort_values('cost of inventory', ascending = False, inplace = True)
    dict_[l] = df

function results
sku         2817
part        1406
not real     160
combo        102
Name: label, dtype: int64


### To Excel

In [13]:
if close_workbook == 'yes':
    workbook_title = 'As of {} - More Than 90 Days of Stock Report'.format(now)
    worksheet_ls = ['part','not real','sku','combo']

    # create workbook
    workbook = xlsxwriter.Workbook(csv_path + workbook_title + '.xlsx', {'nan_inf_to_errors': False})

    # formats
    num = workbook.add_format({'num_format':'#,##0'})
    num2 = workbook.add_format({'num_format':'#,##0.00'})
    per = workbook.add_format({'num_format':'0.0%'})
    dummy = workbook.add_format({'font_size':11})
    mon1 = workbook.add_format({'num_format':'$#,##0.00'})
    mon2 = workbook.add_format({'num_format':'$#,##0'})

    # create worksheets
    for name in worksheet_ls:
        workbook.add_worksheet(name)

    # send to dict    
    my_worksheets = {}
    for worksheet in workbook.worksheets():
        my_worksheets[worksheet.get_name()] = worksheet    

    # for each df, write to its own worksheet    
    for k,v in dict_.items():
        worksheet = my_worksheets[k]
        df = dict_[k].copy()
        df.drop(['sku id','combo'],1,inplace = True)
        df = df.loc[:,(df != 0).any(0)].copy()
        df.sort_values('days of stock', ascending = False, inplace = True)

        try:
            df.drop('timestamp',1,inplace = True)
        except:
            pass

        start_row = 0
        start_col = 0
        for col in range(len(df.columns)):

            c = df.columns[col]

            if c in 'actual stock|quarter sales|days of stock':
                fmt = num
            elif c in 'daily sales':
                fmt = num2
            elif c == 'latest cost per unit':
                fmt = mon1
            elif c == 'cost of inventory':
                fmt = mon2
            else:
                fmt = dummy

            worksheet.write(start_row, start_col + col, df.columns[col])
            worksheet.set_column(start_col + col, start_col + col,
                                 np.max([len(str(x)) for x in df.iloc[:,col]] + [len(df.columns[col])]) + 1)        

            for row in range(len(df)):
                worksheet.write(start_row + 1 + row, start_col + col, df.iloc[row, col], fmt)

        # add in a column filter
        worksheet.autofilter(start_row, start_col, start_row + len(df), start_col + len(df.columns) - 1)

    workbook.close()
    
print('done')

done
